In [ ]:
!pip install scikit-learn scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
#imports
import pandas as pd
from google.colab import drive
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# reads the regression imputed pre processed  file from the drive
df = pd.read_csv('/content/drive/MyDrive/JOB-ML/Binary Classification/regression imputation/pre-processed_regression_DF.csv')

In [ ]:
x = df.iloc[:, 0:-1]
y = df.iloc[:, -1:].values.ravel()


#splits the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

RandomForest (RF)

In [ ]:
#Define the RF model
model = RandomForestClassifier()

In [ ]:
# Fit the RF model on the hyperparamters found during tuning phase
best_rf_model = RandomForestClassifier(n_estimators=200,
                                       criterion='entropy',
                                       max_features='sqrt',
                                       min_samples_split=5)

# Train the best model using the training data
best_rf_model.fit(x_train, y_train)

RandomForestClassifier(criterion='entropy', min_samples_split=5,
                       n_estimators=200)

In [ ]:
# Make predictions on the test data
predictions = best_rf_model.predict(x_test)

# Evaluate the predictions
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

# Print the evaluation metrics
print("Accuracy:", accuracy*100)
print("Precision:", precision*100)
print("Recall:", recall*100)
print("F1-score:", f1*100)

Accuracy: 99.43138330776247
Precision: 98.42690743251572
Recall: 99.06030485819286
F1-score: 98.74259040288116


SVM

Non Linear

In [ ]:
# Creates an instance of the MinMaxScaler
scaler = MinMaxScaler()

x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train.values), columns=x_train.columns)
x_test_scaled = pd.DataFrame(scaler.fit_transform(x_test.values), columns=x_test.columns)

In [ ]:
svm_model = SVC(kernel='rbf')
svm_model.fit(x_train_scaled, y_train)

SVC()

In [ ]:
y_pred = svm_model.predict(x_test_scaled)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
print(accuracy)

0.970726899728448


Linear

In [ ]:
# Define the SVM model
model = LinearSVC(dual=False)

In [ ]:
# Create the LinearSVC model with the best hyperparameters
best_svm_model = LinearSVC(penalty='l1',tol=2.4004455788496022e-11 ,C=3.9728931339630273,loss='squared_hinge' ,dual=False)
# Train the best model using the training data
best_svm_model.fit(x_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=3.9728931339630273, dual=False, penalty='l1',
          tol=2.4004455788496022e-11)

In [ ]:
# Make predictions on the test data
predictions = best_svm_model.predict(x_test_scaled)

# Evaluate the predictions
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.9091471448592088
Precision: 0.7288661632677329
Recall: 0.9504566670284855
F1-score: 0.8250417272384644


ANN

In [ ]:
x = df.iloc[:, 0:-2].values
y = df.iloc[:, -1:].values.ravel()

#splits the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Creates an instance of the MinMaxScaler
scaler = MinMaxScaler()

# Fits the scaler on the training data and transforms it
x_train_scaled = scaler.fit_transform(x_train)

# Fits the scaler on the testing data and transforms it
x_test_scaled = scaler.transform(x_test)

In [ ]:
# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
x_test_tensor = torch.tensor(x_test_scaled , dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# Create a PyTorch DataLoader for training and test sets
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Define the ANN model
class ANN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define hyperparameters
input_size = x_train.shape[1]
hidden_size = 100
num_classes = len(pd.unique(y_train))
learning_rate = 0.001
num_epochs = 5

# Create the ANN model
model = ANN(input_size, hidden_size, num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the training loss after every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/5], Loss: 0.0013
Epoch [2/5], Loss: 0.0001
Epoch [3/5], Loss: 0.0932
Epoch [4/5], Loss: 0.0009
Epoch [5/5], Loss: 0.0005


In [ ]:
# Evaluation on test set
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    labels_cm = []
    predicted_cm =[]
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        labels_cm.extend(labels.tolist())
        predicted_cm.extend(predicted.tolist())

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 98.83%
